# Installing & Imports

In [165]:
conda install pandas==0.24.2

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda/envs/Python3

  added / updated specs:
    - pandas==0.24.2


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         153 KB
    ------------------------------------------------------------
                                           Total:         153 KB

The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-certificates-2019.11.~ --> pkgs/main::ca-certificates-2020.1.1-0
  openssl            conda-forge::openssl-1.1.1d-h516909a_0 --> pkgs/main::openssl-1.1.1d-h7b6447c_4

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi                                       conda-forge --> pkgs/main



certifi-2019.11.28   | 153 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying

In [166]:
conda upgrade --all -y

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [167]:
conda install snorkel==0.9.0 -c conda-forge

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda/envs/Python3

  added / updated specs:
    - snorkel==0.9.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         149 KB

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates     pkgs/main::ca-certificates-2020.1.1-0 --> conda-forge::ca-certificates-2019.11.28-hecc5488_0
  certifi                                         pkgs/main --> conda-forge
  openssl              pkgs/main::openssl-1.1.1d-h7b6447c_4 --> conda-forge::openssl-1.1.1d-h516909a_0



certifi-2019.11.28   | 149 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction

In [168]:
conda install -c conda-forge textblob

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [169]:
from snorkel.labeling import LFAnalysis
from snorkel.labeling import labeling_function
from snorkel.labeling import PandasLFApplier,LabelModel
from snorkel.preprocess import preprocessor
from textblob import TextBlob
import pandas as pd

# Reading & Preparing TSV file 

In [170]:
# Read TSV file
tsv_reviews = pd.read_csv('/project/reviews.tsv', sep='\t', header=0, encoding='unicode_escape')

In [171]:
tsv_reviews.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [172]:
# Extract review and fresh columns
tsv_reviews = pd.DataFrame(tsv_reviews, columns = ['review', 'fresh'])

In [173]:
tsv_reviews.head()

,review,fresh
0,A distinctly gallows take on contemporary fina...,fresh
1,It's an allegory in search of a meaning that n...,rotten
2,... life lived in a bubble in financial dealin...,fresh
3,Continuing along a line introduced in last yea...,fresh
4,... a perverse twist on neorealism...,fresh


In [174]:
tsv_reviews.isnull().sum()

review    5563
fresh        0
dtype: int64

In [175]:
# drop NaN rows in reviews
index_name = tsv_reviews[(tsv_reviews['review'].isnull())].index
tsv_reviews.drop(index_name, inplace= True)

In [176]:
tsv_reviews.isnull().sum()

review    0
fresh     0
dtype: int64

In [177]:
# rename fresh as 1 and rotten as 0
tsv_reviews['fresh'].replace({'fresh':'1', 'rotten':'0'}, inplace = True)

In [178]:
#Rename columns
tsv_reviews.rename(columns={'fresh':'Freshness','review':'Review'},inplace=True)
tsv_reviews.head()

,Review,Freshness
0,A distinctly gallows take on contemporary fina...,1
1,It's an allegory in search of a meaning that n...,0
2,... life lived in a bubble in financial dealin...,1
3,Continuing along a line introduced in last yea...,1
4,... a perverse twist on neorealism...,1


# Reading & Preparing CSV file 

In [179]:
# Read CSV file
csv_reviews= pd.read_csv('/project/rotten_tomatoes_reviews.csv')
csv_reviews.head()


,Freshness,Review
0,1,"Manakamana doesn't answer any questions, yet ..."
1,1,Wilfully offensive and powered by a chest-thu...
2,0,It would be difficult to imagine material mor...
3,0,Despite the gusto its star brings to the role...
4,0,If there was a good idea at the core of this ...


In [180]:
#Swap Freshness and Review 
columns_titles = ["Review","Freshness"]
csv_reviews=csv_reviews.reindex(columns=columns_titles)

csv_reviews.head()

,Review,Freshness
0,"Manakamana doesn't answer any questions, yet ...",1
1,Wilfully offensive and powered by a chest-thu...,1
2,It would be difficult to imagine material mor...,0
3,Despite the gusto its star brings to the role...,0
4,If there was a good idea at the core of this ...,0


# Web Scraping & Preparing scrapped data 

In [181]:
#Read Web Scraping Data Part1 
web_scraping_reviews_part1= pd.read_csv('/project/web_scraping_rotten_tomatoes_part1.csv')
web_scraping_reviews_part1.head()

,Unnamed: 0,Title,Review,Rating
0,0,The Invisible Man,"A bracingly modern #MeToo allegory that, despi...",rotten
1,1,The Invisible Man,"A smart, unexpected delight.",fresh
2,2,The Invisible Man,Whannell uses stillness and empty spaces again...,fresh
3,3,The Invisible Man,"While chock full of relatively good scares, ca...",rotten
4,4,The Invisible Man,"In this heart-stopping update, the 'Gotcha!' m...",fresh


In [182]:
# rename fresh as 1 and rotten as 0
web_scraping_reviews_part1['Rating'].replace({'fresh':'1', 'rotten':'0'}, inplace = True)

#Rename Rating to Review 
web_scraping_reviews_part1.rename(columns={'Rating':'Freshness'},inplace=True)
web_scraping_reviews_part1.head()

# Extract Review and Freshness columns
web_scraping_reviews_part1= pd.DataFrame(web_scraping_reviews_part1, columns = ['Review', 'Freshness'])

In [183]:
web_scraping_reviews_part1.head()

,Review,Freshness
0,"A bracingly modern #MeToo allegory that, despi...",0
1,"A smart, unexpected delight.",1
2,Whannell uses stillness and empty spaces again...,1
3,"While chock full of relatively good scares, ca...",0
4,"In this heart-stopping update, the 'Gotcha!' m...",1


In [184]:
#Read Web Scraping Data Part2
web_scraping_reviews_part2= pd.read_csv('/project/web_scraping_rotten_tomatoes_part2.csv')
web_scraping_reviews_part2.head()

,Unnamed: 0,title,review,rating
0,0,My Hindu Friend (Meu amigo Hindu),Babenco's cinematic farewell isn't perfect by ...,fresh
1,1,My Hindu Friend (Meu amigo Hindu),This is a good film if you are looking for som...,fresh
2,2,My Hindu Friend (Meu amigo Hindu),"My Hindu Friend is a celebration of life, love...",fresh
3,3,My Hindu Friend (Meu amigo Hindu),I wouldn't miss it; it's a film that's more th...,fresh
4,4,My Hindu Friend (Meu amigo Hindu),"...surreal, reflective (though never sentiment...",fresh


In [185]:
# rename fresh as 1 and rotten as 0
web_scraping_reviews_part2['rating'].replace({'fresh':'1', 'rotten':'0'}, inplace = True)

#Rename Rating to Review 
web_scraping_reviews_part2.rename(columns={'rating':'Freshness', 'review':'Review'},inplace=True)
web_scraping_reviews_part2.head()

# Extract Review and Freshness columns
web_scraping_reviews_part2= pd.DataFrame(web_scraping_reviews_part2, columns = ['Review', 'Freshness'])

In [186]:
web_scraping_reviews_part2.head()

,Review,Freshness
0,Babenco's cinematic farewell isn't perfect by ...,1
1,This is a good film if you are looking for som...,1
2,"My Hindu Friend is a celebration of life, love...",1
3,I wouldn't miss it; it's a film that's more th...,1
4,"...surreal, reflective (though never sentiment...",1


# Combining all the data together 

In [187]:
# csv_reviews.info()
# tsv_reviews.info()
# web_scraping_reviews_part1.info()
# web_scraping_reviews_part2.info()



# Concat two files into all_reviews
all_reviews=pd.concat([csv_reviews, tsv_reviews,web_scraping_reviews_part1,web_scraping_reviews_part2],axis=0, sort=False)
all_reviews.head()

,Review,Freshness
0,"Manakamana doesn't answer any questions, yet ...",1
1,Wilfully offensive and powered by a chest-thu...,1
2,It would be difficult to imagine material mor...,0
3,Despite the gusto its star brings to the role...,0
4,If there was a good idea at the core of this ...,0


In [188]:
all_reviews.shape

(534763, 2)

# Split into test and training set 

In [189]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(all_reviews, test_size=0.2)

In [190]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 427810 entries, 261575 to 277770
Data columns (total 2 columns):
Review       427810 non-null object
Freshness    427810 non-null object
dtypes: object(2)
memory usage: 9.8+ MB


In [191]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106953 entries, 309930 to 257597
Data columns (total 2 columns):
Review       106953 non-null object
Freshness    106953 non-null object
dtypes: object(2)
memory usage: 2.4+ MB


In [192]:
#For training set - need to remove the labels (so we have an unlabelled training set)
train = train.drop('Freshness', 1)

In [193]:
train.head()

,Review
261575,Unlike NBC's previous stage-bound live musica...
470469,There have certainly been worse.
365250,"LBJ is frequently funny, and although it fade..."
2499,They've made the most reverent adaptation of ...
239439,"Heder, with his snappy reactions and his brid..."


In [196]:
#From labelled test set, extract a sample to find out about which labelling functions could be written
#Not sure how big the development split_ can be --> take sample of 1000 data points 

development_split = test.sample(1000,random_state=42)
development_split.head()


,Review,Freshness
124965,"While ""Room 237"" sought evidence for its most...",0
291865,"[A] solid piece of CanCon filmmaking, anchore...",1
12351,The second feature by the openly gay Canadian ...,1
275584,Under the sinuous direction of Calin Peter Ne...,1
233129,A formulaic hodge-podge that trades on a cert...,0


In [197]:
#For finding labelling functions: 
development_split

,Review,Freshness
124965,"While ""Room 237"" sought evidence for its most...",0
291865,"[A] solid piece of CanCon filmmaking, anchore...",1
12351,The second feature by the openly gay Canadian ...,1
275584,Under the sinuous direction of Calin Peter Ne...,1
233129,A formulaic hodge-podge that trades on a cert...,0
130685,"Despite a wooden performance by Keanu Reeves,...",1
277327,As regressive as it is well meaning.,0
452867,Skyscraper is so preposterous it avoids becom...,0
218655,Haneke doesn't offer easy solutions or pat em...,1
403671,A little too self-consciously quirky.,0
